# Jul 21st, 2022 (Fig-2 OCs; combined; p=25)

**Motivation**:  <br>

In [ ]:
# HIDE CODE


import os
import sys
from pprint import pprint
from copy import deepcopy as dc
from os.path import join as pjoin
from scipy.ndimage import gaussian_filter
from IPython.display import display, IFrame, HTML

# tmp & extras dir
git_dir = pjoin(os.environ['HOME'], 'Dropbox/git')
extras_dir = pjoin(git_dir, 'jb-Ca-fMRI/_extras')
fig_base_dir = pjoin(git_dir, 'jb-Ca-fMRI/figs')
tmp_dir = pjoin(git_dir, 'jb-Ca-fMRI/tmp')

# GitHub
sys.path.insert(0, pjoin(git_dir, '_Ca-fMRI'))
from figures.fighelper import *
from analysis.final import *
from utils.render import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
%matplotlib inline
set_style()

## Fig dir

In [ ]:
nn = 512
perc = 25
fig_dir = 'Fig2_OCs'
fig_dir = pjoin(fig_base_dir, 'paper2.0', f'n{nn}-p{perc}', fig_dir)
os.makedirs(fig_dir, exist_ok=True)
print(fig_dir)

kws_fig = {
    'transparent': True,
    'bbox_inches': 'tight',
    'dpi': 300,
}

## Prepare (512)

In [ ]:
mice = Mice(nn)
num_k = None

## Get results

In [ ]:
final_perm={4: 5, 5: 4}
final_perm = {
    i: final_perm.get(i, i)
    for i in range(7)
}

In [ ]:
%%time

results = {}
for num_k in [3, 7, 20]:
    res = similiarity_results(
        mice=mice,
        perc=perc,
        num_k=num_k,
        n_resamples=int(1e2),
        merge_thres=0.444 if
        num_k == 7 else None,
        final_perm=final_perm if
        num_k == 7 else None,
    )
    results[num_k] = res

    bt = BootStrap(**res['props_ca'])
    _ = bt.show(res['pi_sorted'], **show_kws(num_k))

## Overall sim

### Box plot

In [ ]:
from figures.fig_ocs import similarity_box as go
fig, _ = go(results, width=0.27)

fig.savefig(pjoin(fig_dir, 'sim_bx.pdf'), **kws_fig)
fig.savefig(pjoin(fig_dir, 'sim_bx.png'), **kws_fig)

### Bar plot

In [ ]:
from figures.fig_ocs import similarity_bar as go
fig, _ = go(results, legend_fontsize=12)

fig.savefig(pjoin(fig_dir, 'sim_br.pdf'), **kws_fig)
fig.savefig(pjoin(fig_dir, 'sim_br.png'), **kws_fig)

## FDR correction

In [ ]:
fig, axes = create_figure(3, 3, (12, 7), 'col')

for i, (num_k, res) in enumerate(results.items()):
    for j, (k, test) in enumerate(res['tests'].items()):
        ax = axes[i, j]
        sns.kdeplot(
            x=test.null_distribution,
            shade=True,
            ax=ax,
        )
        ax.axvline(test.statistic, color='r', ls='-', lw=1.2)
        ax.annotate(
            text=f"p = {test.pvalue:1.1e}",
            xycoords='axes fraction',
            xy=(0.08, 0.85),
        )
        if i == 0:
            ax.set_title(f"{k[0][0]} vs. {k[0][1]}\nVS.\n{k[1][0]} vs. {k[1][1]}", y=1.05)
        if j == 0:
            ax.set_ylabel(f'K = {num_k}')
        else:
            ax.set_ylabel('')
plt.show()

### Significance levels

In [ ]:
pvals = {}
for num_k, res in results.items():
    for k, test in res['tests'].items():
        pvals[(num_k, ) + k] = test.pvalue

In [ ]:
for alpha in [0.001, 0.01, 0.05]:
    reject, *_ = multipletests(
        pvals=list(pvals.values()),
        method='fdr_bh',
        alpha=alpha,
    )
    print(f"alpha = {alpha};  reject:\n{reject}")

## Bonferroni correction

### Significance levels

In [ ]:
pvals = {}
for num_k, res in results.items():
    for k, test in res['tests'].items():
        pvals[(num_k, ) + k] = test.pvalue

In [ ]:
for alpha in [0.001, 0.01, 0.05]:
    reject, *_ = multipletests(
        pvals=list(pvals.values()),
        method='bonferroni',
        alpha=alpha,
    )
    print(f"alpha = {alpha};  reject:\n{reject}")

In [ ]:
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(
    pvals=list(pvals.values()),
    method='bonferroni',
    alpha=0.05,
)
print(f"alpha = {alpha};  reject:\n{reject}")

In [ ]:
np.round(pvals_corrected, 3)

In [ ]:
alphacBonf, 0.05 / 9